## Connessione asincrona

In [ ]:
import time
from ib_async import *
util.startLoop()  # uncomment this line when in a notebook
import numpy as np

ib = IB()
id_connection = float(input('Enter the connection id: '))
lookback_window = int(input('Enter the lookback window: '))
number_sdev = int(input('Enter the number of standard deviations: '))

def connect_ib():
    while not ib.isConnected():
        try:
            ib.connect('127.0.0.1', 4002, clientId=id_connection)
            print("Connected to IB")
        except Exception as e:
            print(f"Connection failed: {e}. Retrying in 60 seconds...")
            time.sleep(60)

connect_ib()

contract = Forex('EURGBP')
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='900 S',
        barSizeSetting='5 secs',
        whatToShow='MIDPOINT',
        useRTH=False,
        formatDate=1,
        keepUpToDate=True)

close_prices = [] #crea una lista
datetime = []
positions_list = []
positions_list.append(0)

def onBarUpdate(bars, hasNewBar):
    #logging features:
    close_prices.append(float(bars[-1].close)) #append il prezzo di chiusura alla lista
    datetime.append(str(bars[-1].date))
    ohlc = [] #crea una lista
    ohlc.append(str(bars[-1].date)) #append la data
    ohlc.append(str(bars[-1].close))

    #calculate the bollinger bands:
    if len(close_prices) >= lookback_window:
        std = np.std(close_prices[-lookback_window:]) #calcola la deviazione standard dei 20 prezzi precedenti
        sma = np.mean(close_prices[-lookback_window:])
        upper_band = sma + number_sdev * std #calcola la banda superiore
        lower_band = sma - number_sdev * std #calcola la banda inferiore
        ohlc.append(str(upper_band))
        ohlc.append(str(lower_band))
        ohlc.append(str(sma))
    else:
        ohlc.append('N/A')  # Not enough data for Bollinger Bands
        positions_list.append(0) #we continue to append 0 to the positions list since we don't have enough data to make a decision
    
    #--------------------------------------------------------------------
    #create strategy's logic:

    #--------------------------------------------------------------------
    accountValue = ib.accountSummary()
    #BUY VALUE
    accountBalanceStr = accountValue[9].value  # Convert the account balance to an integer and save it to a variable
    accountBalanceGBP = round(float(accountBalanceStr)) #GBP ACCOUNT VALUE
    SELL_Value = accountBalanceGBP * 0.1 #10% of the account value

    #SELL VALUE
    accountBalanceEUR = accountBalanceGBP / float(bars[-1].close) #EUR ACCOUNT VALUE
    BUY_Value = accountBalanceEUR * 0.1 #10% of the account value
    #--------------------------------------------------------------------
    
    if len(close_prices) >= lookback_window:
        if positions_list[-1] == 0:
            # ib is not in a position
            if float(bars[-1].close) > float(upper_band):
                positions_list.append(-1)
                ib.placeOrder(contract, MarketOrder('SELL', SELL_Value))
            elif float(bars[-1].close) < float(lower_band):
                positions_list.append(1)
                ib.placeOrder(contract, MarketOrder('BUY', BUY_Value))
        
        else:
            # ib is in a position
            if positions_list[-1] == 1 and float(bars[-1].close) > float(sma):
                positions_list.append(0)
                ib.placeOrder(contract, MarketOrder('SELL', SELL_Value))
            elif positions_list[-1] == -1 and float(bars[-1].close) < float(sma):
                positions_list.append(0)
                ib.placeOrder(contract, MarketOrder('BUY', BUY_Value))
    #--------------------------------------------------------------------

    print(",".join(ohlc), 'current position is:', positions_list[-1]) #printa la lista separata da virgole

bars.updateEvent += onBarUpdate

def check_connection():
    while True:
        if not ib.isConnected():
            print("Disconnected from IB. Attempting to reconnect...")
            connect_ib()
            RestartFunction()


        time.sleep(60)  # Check every 60 seconds

# Run the connection checker in a separate thread
import threading
connection_thread = threading.Thread(target=check_connection)
connection_thread.start()


#we define the restart function
def RestartFunction():
    global bars
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='900 S',
            barSizeSetting='5 secs',
            whatToShow='MIDPOINT',
            useRTH=False,
            formatDate=1,
            keepUpToDate=True)
    
    bars.updateEvent += onBarUpdate
    ib.run()






ib.run()

### Test vari

In [2]:
import ib_async
import time
from ib_async import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=1)
if ib.isConnected():
    print("Connected to IB")

ib.run()

Connected to IB


In [15]:
accountValue = ib.accountSummary()
accountBalanceStr = accountValue[9].value  # Convert the account balance to an integer and save it to a variable
accountBalance = round(float(accountBalanceStr))
print(accountBalance)

1023803


In [ ]:
ib.disconnect()